In [1]:
from neo4j import GraphDatabase

class Persona:
    def __init__(self,nombre,apellido,correo):
        self.nombre=nombre
        self.apellido=apellido
        self.correo=correo

def get_ed(tx):
    comunidad = []
    result = tx.run("MATCH (n) RETURN n")
    for record in result:
        comunidad.append(record["n"])
    return comunidad
        
def obtenerPersonas():
    text =""
    uri = "bolt://35.239.4.175:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"),encrypted=False)
    with driver.session() as session:
        comunidad = session.read_transaction(get_ed)
        for i in range(len(comunidad)):
            #print(comunidad[i].labels)
            text+=comunidad[i]["nombre"]+" "+comunidad[i]["correo"]+"\n"
            #print(comunidad[i]["name"]," -> ",comunidad[i]["apellido"]," -> ", comunidad[i]["edad"])
    driver.close()
    return text

def guardarPersona(per):
    uri = "bolt://35.239.4.175:7687"
    driver = GraphDatabase.driver(uri, auth=("neo4j", "neo4j"),encrypted=False)
    with driver.session() as session:
        session.write_transaction(guardar, per)

def guardar(tx, per):
    result = tx.run("CREATE (p:Persona) "
                    "SET p.nombre = $nombre "
                    "SET p.apellido = $apellido "
                    "SET p.correo = $correo ", nombre=per.nombre, apellido=per.apellido, correo=per.correo)

In [2]:
from clips import Environment, Symbol
def insertar(res,environment):
    fs1 = '(confirmacionp ' + str(res[0]) + ' )'
    fs2 = '(confirmacionm ' + str(res[1]) + ' )'
    fs3 = '(confirmacionr ' + str(res[2]) + ' )'
    fact1 = environment.assert_string(fs1)
    fact2 = environment.assert_string(fs2)
    fact3 = environment.assert_string(fs3)
    
def ejecutar(res):
    environment = Environment()
    environment.load('examen.clp')
    insertar(res,environment)
    environment.run()
    lis=[]
    for fact in environment.facts():
        tt = list(fact)
        for h in tt:
            lis.append(h)
    environment.reset()
    resultado = lis[len(lis)-1]
    if(resultado=="ralto"):
        return "Se recomienda la carrera"
    elif(resultado=="rmedio"):
        return "Se recomienda pero debe estudiar en lo que falla"
    elif(resultado=="rbajo"):
        return "Si toma la carrera debera estudiar más duro"

In [3]:
import json
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_cloud_sdk_core.api_exception import ApiException
from ibm_watson import LanguageTranslatorV3
from ibm_watson import PersonalityInsightsV3

authenticator = IAMAuthenticator('vh0j48-6QDFVQy6-qehepHiOX_8Dc8_3anwEFFJ-7CRr')
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=authenticator
)

language_translator.set_service_url('https://api.us-south.language-translator.watson.cloud.ibm.com/instances/590a49d1-ee08-4611-9f75-62ae74ee0d7c')

authenticator = IAMAuthenticator('RzHAgFvAbd4YJn3-21N8p6YehshWsMeLdlGqa3hmSmuR')
assistant = AssistantV2(
    version='2020-04-01',
    authenticator=authenticator
)

assistant.set_service_url('https://api.us-south.assistant.watson.cloud.ibm.com/instances/9276aae1-9178-4219-ac80-ec84e0124427')
assistant.set_disable_ssl_verification(False)

session = assistant.create_session( assistant_id='c4f40484-71ea-4e58-95c2-90470521dd8a').get_result()
print(session)

authenticator = IAMAuthenticator('PXtrxiU-I5ny32L3k8HPcmzwPvb4bdJFcslA3oIG-48J')
personality_insights = PersonalityInsightsV3(
    version='2017-10-13',
    authenticator=authenticator
)
personality_insights.set_disable_ssl_verification(False)
personality_insights.set_service_url('https://api.us-south.personality-insights.watson.cloud.ibm.com/instances/68f2775d-8ee5-422f-b4a9-7d7d01168f43')

{'session_id': '3258341c-fe2c-4eaf-b04e-0f1efaccfd73'}


In [4]:
import urllib3
def identLanguage(mensaje):
    language = language_translator.identify(
        mensaje).get_result()
    r= json.dumps(language, indent=2)
    rp= json.loads(r)
    return rp["languages"][0]["language"]

def traducir(mensaje):
    leng=identLanguage(mensaje)
    print("LENguaje Broma ->" ,leng)
    if(leng == "fr" or leng=="ca" or leng=="en"):
        print("Idioma: ",leng)
        translation = language_translator.translate(
        text=mensaje,
        model_id=leng+"-es").get_result()
        r= json.dumps(translation, indent=2)
        rp= json.loads(r)
        return rp["translations"][0]["translation"],leng
    else:
        return mensaje,"es"
    
def traducir2(mensaje,leng):
    print("Lenguaje +",leng)
    if(leng =="fr" or leng=="ca" or leng=="en"):
        print("IAT:",leng)
        translation = language_translator.translate(
        text=mensaje,
        model_id="es-"+leng).get_result()
        r= json.dumps(translation, indent=2)
        rp= json.loads(r)
        return rp["translations"][0]["translation"]
    else:
        return mensaje

def sendMessageIBM(mensaje):
    print(mensaje)
    trad=traducir(mensaje)
    message = assistant.message('c4f40484-71ea-4e58-95c2-90470521dd8a',
        session["session_id"],
        input={'text': trad[0]},
        context={
            'metadata': {
                'deployment': 'myDeployment'
            }
        }).get_result()
    r=json.dumps(message, indent=2)
    rp=json.loads(r)
    return traducir2(rp['output']['generic'][0]['text'],trad[1])
def analizar(url):
    http = urllib3.PoolManager()
    response = http.request('GET', url)
    res=""
    try:
        data = response.data.decode('utf-8')
        profile = personality_insights.profile(
        data,
        'application/json',
        content_type='text/plain',
        consumption_preferences=True,
        content_language=identLanguage(data),
        accept_language="es",
        raw_scores=True
        ).get_result()
        r=json.dumps(profile, indent=2)
        rp=json.loads(r)
        res+="PALABRAS ANALIZADAS: "+str(rp["word_count"])+"\n"
        res+="RESULTADOS DE PERSONALIDAD\n"
        for i in rp['personality']:
            res+="Personalidad: "+i["name"]+"\n"
            for j in i["children"]:
                res+="Característica: "+j["name"]+" : "+"{}%".format(int(round(j["percentile"],2)*100))+"\n"
        res+="RESULTADOS DE NECESIDADES\n"
        for i in rp['needs']:
            res+="Necesidad: "+i["name"]+" : "+"{}%".format(int(round(i["percentile"],2)*100))+"\n"
        res+="RESULTADOS DE VALORES\n"
        for i in rp['values']:
            res+="Valor: "+i["name"]+" : "+"{}%".format(int(round(i["percentile"],2)*100))+"\n"
        return res
    except UnicodeDecodeError:
        return "No archivo de texto"
    except ApiException:
        return "Menos de las palabras minimo 100"

In [ ]:
import random
from flask import Flask, request
from pymessenger.bot import Bot

app = Flask(__name__)
ACCESS_TOKEN = 'EAAJccYUYmiEBAMXdQjkCtkJ1RsPKuNSzZAU60IHZAwZBNiBJ7KRlGLPC1zfTjLsdQlBBQ3lZCDEYXy1aDuYZAX0pIVjZBRf0m67fZA6fNSU9gnDWF1D9GpxUM7WgcpLs45DLzQtOGQOGE55bctN6gWAGprz6sNLicmzNvIQyjb4kAZDZD'
VERIFY_TOKEN = 'VERIFY_TOKEN'
bot = Bot(ACCESS_TOKEN)

#We will receive messages that Facebook sends our bot at this endpoint 
@app.route("/", methods=['GET', 'POST'])
def receive_message():
    if request.method == 'GET':
        """Before allowing people to message your bot, Facebook has implemented a verify token
        that confirms all requests that your bot receives came from Facebook.""" 
        token_sent = request.args.get("hub.verify_token")
        return verify_fb_token(token_sent)
    #if the request was not get, it must be POST and we can just proceed with sending a message back to user
    else:
        # get whatever message a user sent the bot
       output = request.get_json()
       for event in output['entry']:
          messaging = event['messaging']
          for message in messaging:
            if message.get('message'):
                #Facebook Messenger ID for user so we know where to send response back to
                recipient_id = message['sender']['id']
                if message['message'].get('text'):
                    text=message['message'].get('text')
                    response_sent_text = get_message(recipient_id,text)
                    send_message(recipient_id, response_sent_text)
                #if user sends us a GIF, photo,video, or ny other non-text item
                #if message['message'].get('attachments'):
                    #archivo = message['message'].get('attachments')
                    #tipo = archivo[0]["type"]
                    #url=archivo[0]["payload"]["url"]
                    #response_sent_nontext=""
                    #if(tipo=="file"):
                    #    response_sent_nontext=analizar(url)
                    #else:
                    #    response_sent_nontext = "No es un archivo de texto"
                    #send_message(recipient_id, response_sent_nontext)
    return "Message Processed"

def verify_fb_token(token_sent):
    #take token sent by facebook and verify it matches the verify token you sent
    #if they match, allow the request, else return an error 
    if token_sent == VERIFY_TOKEN:
        return request.args.get("hub.challenge")
    return 'Invalid verification token'

#chooses a random message to send to the user
def get_message(rid,text):
    res=""
    if("si " in text or "no " in text):
        lista = text.split(" ")
        print(lista)
        send_message(rid,ejecutar(lista))
    else:
        res=sendMessageIBM(text)
        if(" " in text and "@" in text):
            per= text.split(" ")
            if(len(per)==3):
                persona=Persona(per[0],per[1],per[2])
                guardarPersona(persona)
            else:
                res="Mal proceso de registro intente nuevamente"      
    return res
#uses PyMessenger to send response to user
def send_message(recipient_id, response):
    #sends user the text message provided via input response parameter
    bot.send_text_message(recipient_id, response)
    return "success"
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


hola
LENguaje Broma -> es
Lenguaje + es


127.0.0.1 - - [05/Aug/2020 14:03:57] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 14:03:58] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 14:03:58] "POST / HTTP/1.1" 200 -


análisis
LENguaje Broma -> hu
Lenguaje + es


...

127.0.0.1 - - [05/Aug/2020 14:06:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 14:06:27] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 14:06:27] "POST / HTTP/1.1" 200 -


registro
LENguaje Broma -> eo
Lenguaje + es


127.0.0.1 - - [05/Aug/2020 14:08:21] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 14:08:22] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 14:08:22] "POST / HTTP/1.1" 200 -


Jose Guamán joseg@micorreo.com
LENguaje Broma -> es
Lenguaje + es


127.0.0.1 - - [05/Aug/2020 14:08:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 14:08:41] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [05/Aug/2020 14:08:41] "POST / HTTP/1.1" 200 -
